# Build a SageMaker Pipeline
在這個部分，我們將一步步構建一個完整的 [SageMaker Pipeline](https://aws.amazon.com/sagemaker/pipelines/)，實現從模型訓練到部署的自動化流程。

<img src="../imgs/pipeline.png" alt="pipeline" width="400"/>



Pipeline 將涵蓋從模型的訓練到部署的完整流程。具體來說，我們將進行以下操作：

1. **TraingModel Step**: 使用 QLoRA fine-tune Phi-3.5-mini 模型，實現模型的定製化訓練。
2. **RegisterModel Step**: 將訓練完成的模型註冊到 SageMaker 的 Model Registry，方便後續管理與版本控制。
3. **LambdaDeployModel Step**: 使用 Lambda 來部署模型到 SageMaker Endpoint，讓模型能夠即時提供服務，並將 SageMaker Endpoint 與 API Gateway 整合，提供內部人員端點以取得模型推論結果。
4. **CreateStreamingResponseLambdaFunction Step**: 創建一個 Lambda Function，通過 FastAPI 和 Lambda Web Adapter 實現流式回應，幫助處理來自 SageMaker Endpoint 的模型推論結果。
5. **CreateLambdaFunctionURL Step**: 為流式回應的 Lambda Function 創建專屬的 Lambda Function URL，讓外部客戶端可以直接通過 URL 獲取流式數據。

透過這個部分，你將實踐 MLOps 的任務，學習如何使用 SageMaker Pipeline 實現機器學習模型的訓練、自動化部署，以及如何將 Lambda 與 SageMaker 整合，提供流式回應 (Streaming Response) 給應用程式使用。


<img src="../imgs/streaming-response.gif" alt="pipeline" width="400"/>

_API Reference: https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#steps_

## 配置環境

在這個步驟，我們將安裝兩個重要的 Python 套件，這些是我們在整個 SageMaker Pipeline 中使用的核心工具：

1. **sagemaker**: AWS 提供的官方 [SageMaker SDK](https://sagemaker.readthedocs.io/en/stable/)，它能讓我們在 Jupyter Notebook 中直接與 SageMaker 互動，用來建立和管理訓練作業、模型部署等。
2. **transformers**: 這是一個來自 [Hugging Face 的庫](https://huggingface.co/docs/transformers/index)，專門用於自然語言處理（NLP）模型的訓練與使用。在這個工作坊中，我們會使用它來 fine-tune 預訓練模型。

> 我們將指定安裝 `transformers==4.44.2` 版本來確保與範例代碼的相容性。

執行這段程式碼後，你的環境會安裝這些必要的套件，並準備好進行接下來的操作。

In [ ]:
!pip install sagemaker transformers==4.44.2 --quiet

在這個步驟中，我們將進行 SageMaker 環境的初始化工作。這些操作對於接下來的 SageMaker Pipeline 非常重要，因為它們確保我們有正確的權限與資源來運行所有的步驟。

以下是這段程式碼的核心功能：
1. **引入所需的模組**: 我們導入了用於構建 SageMaker Pipeline 的一系列模組，包括 `TrainingStep`、`ProcessingStep`、`CreateModelStep` 和 `ParameterString` 等，這些將幫助我們定義和管理 Pipeline 中的各個步驟。
2. **設定 SageMaker Session**: 使用 `sagemaker.Session()` 來啟動一個 SageMaker Session，這個 session 是我們與 SageMaker 互動的橋樑。我們還設置了 S3 bucket 作為存放模型和資料的默認位置。如果沒有指定 bucket，則使用 SageMaker 的默認 bucket。
3. **獲取執行角色 (Execution Role)**: 我們使用 `sagemaker.get_execution_role()` 來獲取執行 SageMaker 操作所需的 IAM 角色，這個角色賦予必要的權限來執行訓練和部署任務。如果自動獲取角色失敗，會手動指定一個預設的 SageMaker 執行角色。

In [ ]:
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import TrainingStep, ProcessingStep, CreateModelStep, CacheConfig
from sagemaker.workflow.parameters import ParameterString
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.model_step import ModelStep
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import boto3

sess = sagemaker.Session()

sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

## Pipeline Parameters

在建立 Pipeline 時，我們可以定義一些參數，以便每次執行時，可以動態地傳入一些配置以便根據不同的需求進行設定與調整，這些參數要在建立 Pipeline 時定義。

<img src="../imgs/pipeline-parameters.png" alt="pipeline-parameters" width="400"/>

1. **定義 Pipeline 參數**:
   - `inference_instance_type`: 這個參數定義了推理所需的實例類型，默認值為 `"ml.g5.xlarge"`，這是一個高性能的 GPU 實例，適合處理大型模型的推論工作。
   - `model_artifact_s3_uri`: 這個參數定義了模型檔案的 S3 路徑位置。

2. **配置其他 Pipeline 設定**:
   - `model_package_group_name`: 這裡定義了模型註冊所屬的群組名稱，我們將其設為 `"Demo-SageMaker-Pipeline-Group"`，便於後續在 Model Registry 中進行模型版本管理。
   - `cache_config`: 這是 SageMaker Pipeline 中的快取配置。通過啟用快取，我們可以在相同的步驟沒有變化時跳過重複運行，從而加快 pipeline 的執行速度。在此例中，我們設置快取的有效期限為 30 天（`"30d"`），也就是在這段時間內，如果該步驟的輸入沒有變化，將不會重新運行。

這些參數與配置為我們的 pipeline 提供了靈活性和效率，讓模型訓練與推理可以根據需求動態調整，同時提升了 pipeline 的運行效能。

_API Referecne: https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#parameters_

In [ ]:
# 定義參數
inference_instance_type = ParameterString(name="InferenceInstanceType", default_value="ml.g5.xlarge")
model_artifact_s3_uri = ParameterString(name="ModelArtifactS3Uri")
model_name = ParameterString(name="ModelName", default_value="psy-1-model-created-at-2024-")
random_string = ParameterString(name="RandomString")

# 其他配置
model_package_group_name = "Demo-SageMaker-Pipeline-Group"

## 定義 Register Model Step

### 什麼是 Model Registry？

在定義前，我們先來了解一下 [Model Registry](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry.html) 是什麼。

Model Registry 是 SageMaker 提供的一個功能，幫助我們管理和追蹤機器學習模型的不同版本。當我們在模型訓練完成後，經常會需要對模型進行版本控制、審核與部署，Model Registry 正是為這些需求設計的。透過 Model Registry，我們可以：
- **模型版本控制**：每次訓練完成後，可以將模型註冊到 Model Registry 中，這樣我們可以方便地管理多個版本的模型。
- **審核與批准流程**：模型可以設置不同的狀態，例如 Pending、Approved 或 Rejected，幫助我們管理模型的生命週期。
- **便於部署**：一旦模型被註冊，我們可以快速將它部署到 SageMaker Endpoint，並且利用不同實例來處理推理請求。

### 使用 RegisterModel Step

Document: https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-register-model

這段程式碼展示了如何在 SageMaker Pipeline 中使用 `RegisterModel` 步驟，將訓練完成的模型註冊到 Model Registry 中。

1. **取得 LLM 映像 URI**：
   - 我們使用 `get_huggingface_llm_image_uri()` 來獲取 Hugging Face 大型語言模型 (LLM) 的 Image URI，這是用來推理時的 Docker Image，確保模型可以在 SageMaker Endpoint 上正確運行。

2. **定義模型配置**：
   - `config` 變數定義了推理過程中的一些環境配置，比如最大輸入長度（`MAX_INPUT_LENGTH`）和使用的 GPU 數量（`SM_NUM_GPUS`）。這些配置將在模型註冊時被應用，確保推理環境和訓練時的條件一致。

3. **創建 Hugging Face 模型**：
   - 使用 `HuggingFaceModel` 來將訓練好的模型包裝成一個 SageMaker 模型。這裡，我們通過 `train_step.properties.ModelArtifacts.S3ModelArtifacts` 來指定模型的 S3 路徑，這個路徑是訓練步驟中產生的模型檔案存放位置。
   - 我們還指定了推理時需要的 Docker 映像 URI、執行角色、SageMaker Session 和環境變數。

4. **註冊模型**：
   - 使用 `RegisterModel` 步驟將模型註冊到 Model Registry 中。這裡，我們指定了模型的 `content_types` 和 `response_types` 來定義模型可以處理的輸入與輸出格式（如 `application/json`），並且定義了推理實例類型（`inference_instance_type`）。
   - 我們還將模型註冊到一個名為 `"Demo-SageMaker-Pipeline-Group"` 的模型包群組中，這有助於進行版本管理。最後，將 `approval_status` 設為 `"Approved"`，表示這個模型是已審核通過的，可以直接部署。

透過這個步驟，我們可以將訓練完成的模型保存到 SageMaker 的 Model Registry 中，方便進行版本控制與部署。這是 MLOps 的一個重要部分，能確保我們的模型管理更加有條理且可追蹤。


In [ ]:
import json

from sagemaker.huggingface import HuggingFaceModel
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  session=sess,
)

config = {
    'HF_MODEL_ID': "/opt/ml/model", # path to where sagemaker stores the model
    'SM_NUM_GPUS': json.dumps(1), # Number of GPU used per replica
    'MAX_INPUT_LENGTH': json.dumps(1024), # Max length of input text
    'MAX_TOTAL_TOKENS': json.dumps(2048), # Max length of the generation (including input text)
}

model = HuggingFaceModel( # https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#hugging-face-model
    model_data={'S3DataSource':{'S3Uri': model_artifact_s3_uri,'S3DataType': 'S3Prefix','CompressionType': 'None'}}, # https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html
    role=role,
    image_uri=llm_image,
    sagemaker_session=sess,
    env=config
)

register_step = RegisterModel(
    name="RegisterModel",
    model=model,
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=[inference_instance_type],
    model_package_group_name=model_package_group_name,
    approval_status="Approved",
)

## 定義 Lambda Step for Deploying model endpoint

接下來我們要來定義 Lambda Step，這個步驟將會使用 Lambda Function 來部署模型到 SageMaker Endpoint，讓模型能夠即時提供服務。
我們會先撰寫一個 `lambda_deployer.py` 檔案，這個檔案將會被 Lambda Function 使用

documents: 
- https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-lambda
- https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_model.html

### Lambda 部署模型與 API Gateway 創建

在這段程式碼中，我們定義了一個 Lambda function，目的是將 SageMaker 模型部署到 SageMaker Endpoint 並自動創建一個 API Gateway，讓外部用戶端可以通過 API 調用這個端點。

#### 主要步驟如下：

1. **Lambda Function 輸入參數**：
   - Lambda 使用 `event` 中傳入的參數，包括模型的 ARN、端點名稱、角色 ARN 等，以動態設定模型部署的細節。

2. **創建 SageMaker 模型**：
   - 使用 `boto3` 的 `sagemaker` 客戶端，調用 `create_model()` 函數來創建一個新的 SageMaker 模型。這個模型會根據提供的 `model_package_arn` 和 IAM 執行角色進行配置。

3. **創建 Endpoint 配置與 Endpoint**：
   - `create_endpoint_config()` 創建端點的配置，定義了 SageMaker 如何處理流量的實例類型與數量。
   - 然後，透過 `create_endpoint()` 創建實際的 SageMaker Endpoint，該端點將用來進行模型推理。

4. **創建 API Gateway**：
   - 使用 `apigateway_client` 創建一個新的 API Gateway，並定義它將會處理的 HTTP 請求方法（POST）。API Gateway 將通過 SageMaker Runtime API 來調用 SageMaker 端點，實現與模型的整合。
   - 通過 `put_integration()` 函數，我們將 API Gateway 與 SageMaker Runtime 進行整合，這使得 API Gateway 能夠直接向 SageMaker 發送推理請求。

5. **部署 API Gateway**：
   - 創建 API Gateway 的資源、方法和整合配置後，透過 `create_deployment()` 將 API 部署到 `"dev"` 階段，並生成 API 的可調用 URL。

6. **返回結果**：
   - Lambda 最後返回 API Gateway 的 URL 及端點名稱，表示模型部署和 API Gateway 創建成功。


In [ ]:
%%writefile lambda_deployer.py

import json

import boto3

import time

# Use the current time to define unique names for the resources created
current_time = time.strftime("%H-%M", time.localtime())


def lambda_handler(event, context):
    """Lambda function to deploy a model to an Endpoint using boto3 and create an API Gateway for SageMaker"""

    # 使用 event 中的參數
    random_string = event["random_string"]
    model_package_arn = event["model_package_arn"]
    model_name = event["model_name"] + "-run-at-" + current_time + "-" + random_string
    endpoint_name = event["endpoint_name"]
    endpoint_config_name = endpoint_name + "-" + current_time + "-" + random_string
    role = event["role"]
    apigateway_role = event["apigateway_role"]
    inference_instance_type = event["inference_instance_type"]

    sm_client = boto3.client("sagemaker")
    apigateway_client = boto3.client("apigateway")

    # 創建 SageMaker 模型
    create_model_response = sm_client.create_model(
        ModelName=model_name,
        PrimaryContainer={"ModelPackageName": model_package_arn},
        ExecutionRoleArn=role,
    )

    # 創建端點配置
    create_endpoint_config_response = sm_client.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[
            {
                "InstanceType": inference_instance_type,
                "InitialVariantWeight": 1,
                "InitialInstanceCount": 1,
                "ModelName": model_name,
                "VariantName": "AllTraffic",
            }
        ],
    )

    # 創建端點
    create_endpoint_response = sm_client.create_endpoint(
        EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
    )

    # 創建 API Gateway
    api_name = f"{model_name}-api"
    create_api_response = apigateway_client.create_rest_api(
        name=api_name,
        description=f"API Gateway for SageMaker endpoint {endpoint_name}",
        endpointConfiguration={"types": ["REGIONAL"]},
    )

    # 取得 API Gateway 的根資源 ID
    api_id = create_api_response["id"]
    root_id = apigateway_client.get_resources(restApiId=api_id)["items"][0]["id"]

    # 創建 POST 方法並設定
    apigateway_client.put_method(
        restApiId=api_id,
        resourceId=root_id,
        httpMethod="POST",
        authorizationType="NONE",
    )

    # 設置 SageMaker Runtime 與 API Gateway 的整合
    apigateway_client.put_integration(
        restApiId=api_id,
        resourceId=root_id,
        httpMethod="POST",
        type="AWS",  # 使用 AWS Service Integration
        integrationHttpMethod="POST",
        uri=f"arn:aws:apigateway:{boto3.Session().region_name}:runtime.sagemaker:path/endpoints/{endpoint_name}/invocations",
        credentials=apigateway_role,  # 指定具有 SageMaker InvokeEndpoint 權限的角色
    )

    # 設置 Integration Response，以便 API Gateway 正確處理 SageMaker 回應
    apigateway_client.put_integration_response(
        restApiId=api_id,
        resourceId=root_id,
        httpMethod="POST",
        statusCode="200",
        responseTemplates={"application/json": "$input.body"},
    )

    # 創建方法回應，確保有適當的狀態碼
    apigateway_client.put_method_response(
        restApiId=api_id,
        resourceId=root_id,
        httpMethod="POST",
        statusCode="200",
        responseModels={"application/json": "Empty"},
    )

    # 部署 API Gateway
    apigateway_client.create_deployment(
        restApiId=api_id,
        stageName="dev",
    )

    # 返回 API Gateway 的 URL
    api_url = (
        f"https://{api_id}.execute-api.{boto3.Session().region_name}.amazonaws.com/dev"
    )

    return {
        "statusCode": 200,
        "body": json.dumps(
            {
                "message": f"Created Endpoint {endpoint_name} and API Gateway {api_name}!",
                "endpoint_name": endpoint_name,
                "api_url": api_url,
            }
        ),
    }


### 建立 Lambda 和 API Gateway 所需的 IAM Role

在這段程式碼中，我們定義了一些用來建立和配置 IAM 角色 (Role) 的函數，這些角色將會賦予 Lambda 和 API Gateway 所需的權限，讓它們能夠調用 SageMaker 端點並執行必要的操作。

#### 這段程式碼的目的是：
- 自動化 IAM 角色的創建與配置，確保 Lambda 和 API Gateway 擁有正確的權限來進行 SageMaker 端點的部署和調用。
- 避免重複附加策略，確保 IAM 角色的權限管理保持簡潔而不冗餘。

透過這些 IAM 角色的設定，我們能夠確保模型的部署與推理請求能順利執行，並且滿足各個服務的權限要求。


In [ ]:
import boto3
import json

iam = boto3.client('iam')

def attach_policy_if_missing(role_name, policy_arn):
    """檢查並附加策略到角色，若策略已存在則跳過"""
    try:
        # 獲取附加到角色的現有策略
        attached_policies = iam.list_attached_role_policies(RoleName=role_name)['AttachedPolicies']
        attached_policy_arns = [policy['PolicyArn'] for policy in attached_policies]

        # 如果策略不在已附加列表中，則附加
        if policy_arn not in attached_policy_arns:
            iam.attach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
            print(f"Attached policy {policy_arn} to role {role_name}")
        else:
            print(f"Policy {policy_arn} already attached to role {role_name}")

    except Exception as e:
        print(f"Error attaching policy {policy_arn} to role {role_name}: {e}")

def create_lambda_role(role_name, apigateway_role_arn):
    try:
        # 創建 IAM 角色
        response = iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps({
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "lambda.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            }),
            Description='Role for Lambda to interact with SageMaker, API Gateway, and Bedrock'
        )

        role_arn = response['Role']['Arn']

    except iam.exceptions.EntityAlreadyExistsException:
        print(f'Using existing role: {role_name}')
        response = iam.get_role(RoleName=role_name)
        role_arn = response['Role']['Arn']

    # 無論角色是否已存在，都會檢查並附加所需的策略
    attach_policy_if_missing(role_name, 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')
    attach_policy_if_missing(role_name, 'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess')
    attach_policy_if_missing(role_name, 'arn:aws:iam::aws:policy/AmazonAPIGatewayAdministrator')

    # 附加 Lambda Function URL 創建的策略
    policy_document_url_config = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "lambda:CreateFunctionUrlConfig",
                "Resource": f"arn:aws:lambda:*:{role_arn.split(':')[4]}:function:*"
            }
        ]
    }
    
    iam.put_role_policy(
        RoleName=role_name,
        PolicyName="LambdaURLConfigPolicy",
        PolicyDocument=json.dumps(policy_document_url_config)
    )

    # 在 Lambda 角色中附加 iam:PassRole 權限，允許傳遞 API Gateway 的角色
    policy_document_pass_role = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "iam:PassRole",
                "Resource": apigateway_role_arn
            }
        ]
    }

    iam.put_role_policy(
        RoleName=role_name,
        PolicyName="PassRolePolicy",
        PolicyDocument=json.dumps(policy_document_pass_role)
    )

    # 附加 Bedrock 權限
    bedrock_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock:InvokeModel",
                    "bedrock:ListModels",
                    "bedrock:InvokeModelWithResponseStream"
                ],
                "Resource": "*"
            }
        ]
    }

    iam.put_role_policy(
        RoleName=role_name,
        PolicyName="BedrockInvokePolicy",
        PolicyDocument=json.dumps(bedrock_policy_document)
    )

    print(f"Attached iam:PassRole, lambda:CreateFunctionUrlConfig, and Bedrock policies to {role_name}")

    return role_arn


def create_apigateway_role(role_name):
    try:
        # 創建 API Gateway 用的 IAM 角色
        response = iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps({
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "apigateway.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            }),
            Description='Role for API Gateway to invoke SageMaker Endpoints'
        )

        role_arn = response['Role']['Arn']

    except iam.exceptions.EntityAlreadyExistsException:
        print(f'Using existing role: {role_name}')
        response = iam.get_role(RoleName=role_name)
        role_arn = response['Role']['Arn']

    # 附加允許 API Gateway 調用 SageMaker 的策略
    attach_policy_if_missing(role_name, 'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess')

    return role_arn

### 定義 LambdaDeployModelEndpoint Step

這段程式碼展示了如何將模型的部署過程納入 SageMaker Pipeline，並且使用 Lambda 來自動化模型的部署、端點的創建以及 API Gateway 的設定。Lambda Step 是 SageMaker Pipeline 中一個靈活的步驟，它允許我們在 Pipeline 中插入自定義的 Lambda function，來完成一些 SageMaker 本身無法直接處理的操作。

![apigw-sagemaker-endpoint](../imgs/apigw-sagemaker-endpoint.png)

#### 主要功能和步驟如下：

1. **建立所需的 IAM 角色**：
   - 首先，我們使用 `create_apigateway_role()` 和 `create_lambda_role()` 來創建 API Gateway 和 Lambda 所需的 IAM 角色，這些角色賦予它們調用 SageMaker 端點的權限。

2. **定義 Lambda 相關資源名稱**：
   - 透過 `time.strftime()` 獲取當前時間，並將其添加到資源名稱中，確保每次創建的模型、端點、配置和 Lambda 函數都有獨特的名稱，避免命名衝突。

3. **Lambda 建立與配置**：
   - 使用 `sagemaker.lambda_helper.Lambda` 來定義 Lambda function，這個函數將運行 `lambda_deployer.py` 腳本中的邏輯，負責創建 SageMaker 模型、端點、以及 API Gateway。
   - 我們設置了 Lambda 的基本配置，如執行角色 (`execution_role_arn`)、處理器 (`handler`)、腳本 (`script`)、記憶體大小 (`memory_size`) 和超時時間 (`timeout`)，確保它能夠順利運行所有需要的步驟。

4. **捕捉 Lambda 的輸出**：
   - 使用 `LambdaOutput` 來捕捉 Lambda function 的輸出，這些輸出會返回給 Pipeline。每個 `LambdaOutput` 對應 Lambda 返回的字典中的一個鍵值，這裡我們捕捉了 `statusCode` 和 `body` 這兩個參數，來確認 Lambda function 的執行狀況。

5. **定義 LambdaStep 部署模型端點**：
   - `LambdaStep` 是 SageMaker Pipeline 中用來調用 Lambda function 的核心部分。在這裡，我們將 LambdaStep 設置為名為 `"LambdaStepDeployModelEndpoint"` 的步驟，並傳入所需的 `inputs`，例如模型的 ARN、模型名稱、端點配置名稱、端點名稱以及 Lambda 和 API Gateway 的角色。
   - `inputs` 是通過 `event` 對象傳入到 Lambda function 中，這樣 Lambda 就可以根據這些輸入動態創建 SageMaker 模型和端點，並配置 API Gateway。

### 這段程式碼的目的：
- 使用 Lambda function 自動化模型的部署過程，這包括創建模型、配置端點、並生成 API Gateway 來讓外部系統調用 SageMaker 模型進行推理。
- 利用 `LambdaStep` 將 Lambda function 納入 SageMaker Pipeline 中。

In [ ]:
import time
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
from sagemaker.workflow.functions import JsonGet, Join


apigateway_role = create_apigateway_role("apigateway-role")
lambda_role = create_lambda_role("lambda-deployment-role", apigateway_role_arn=apigateway_role)


# Use the current time to define unique names for the resources created
current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())

# YOU CAN NOT concatenate the pipeline variables using Python primitives 
# https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_building_pipeline.html#not-all-built-in-python-operations-can-be-applied-to-parameters
# Instead, please use sagemaker.workflow.functions.Join(), for more details, visit: https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.functions.Join
endpoint_name = "pipeline-endpoint"

# Lambda helper class can be used to create the Lambda function
func = Lambda(
    function_name="endpoint-deployer",
    execution_role_arn=lambda_role,
    script="lambda_deployer.py",
    handler="lambda_deployer.lambda_handler",
    timeout=600,
    memory_size=10240,
)

# The dictionary retured by the Lambda function is captured by LambdaOutput, each key in the dictionary corresponds to a
# LambdaOutput

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)

# The inputs provided to the Lambda function can be retrieved via the `event` object within the `lambda_handler` function
# in the Lambda
lambda_deploy_step = LambdaStep(
    name="LambdaStepDeployModelEndpoint",
    lambda_func=func,
    inputs={
        "model_package_arn": register_step.properties.ModelPackageArn,
        "model_name": model_name,
        "endpoint_name": endpoint_name,
        "random_string": random_string,
        "role": role,
        "apigateway_role": apigateway_role,
        "inference_instance_type": inference_instance_type
    },
    outputs=[output_param_1, output_param_2])


## 定義 CreateStreamingResponseLambdaFunction Step

我們將創建一個 Lambda Function，這個 Lambda 使用 [Lambda Web Adapter](https://github.com/awslabs/aws-lambda-web-adapter) 結合 FastAPI 作為框架。其核心業務邏輯是利用 `boto3` 調用 `sagemaker_runtime.invoke_endpoint_with_response_stream`，從 SageMaker 端點獲取推理結果，並通過流式回應 (Streaming Response) 的方式將結果回傳給調用 Lambda Function 的用戶端。

![lambda-web-adatper](../imgs/lambda-web-adapter.png)

### Streaming Response Lambda Function 的初始化

在這段程式碼中，我們使用 FastAPI 來構建一個 Lambda Function，它能夠處理用戶端發送的推理請求，並使用流式回應（Streaming Response）的方式將結果返回。這個 Lambda Function 可以與 SageMaker 端點和 Bedrock 服務進行整合，實現流式處理大規模模型的回應。

#### 主要功能包括：
1. **FastAPI 框架**: 我們使用 FastAPI 來設置 HTTP 路由和處理推理請求。這是一個輕量且高效的 Python Web 框架，特別適合在 Lambda 上運行。
2. **CORS 設定**: 由於會使用到瀏覽器來跨域請求，這裡配置了 CORS (Cross-Origin Resource Sharing) 允許來自不同網域的請求。
3. **前端 Demo 頁面配置**: 我們將前端 Demo 頁面掛載到 `/demo` 路徑，並設置一個根路由來將請求重定向到這個頁面。
4. **Bedrock 與 SageMaker 整合**: 這段程式碼定義了如何將用戶端的請求轉換為 Bedrock 和 SageMaker 端點的推理請求，並使用流式回應的方式將模型生成的結果回傳給用戶端。


In [ ]:
%%writefile main.py

import json
import os
from typing import List, Optional

import boto3
import uvicorn
from fastapi import FastAPI
from fastapi.staticfiles import StaticFiles
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import RedirectResponse, StreamingResponse
from pydantic import BaseModel


app = FastAPI()

bedrock = boto3.client("bedrock-runtime")
sagemaker_runtime = boto3.client("sagemaker-runtime")

SAGEMAKER_ENDPOINT_NAME = os.getenv("SAGEMAKER_ENDPOINT_NAME")

# CORS 設定
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

app.mount("/demo", StaticFiles(directory="static", html=True))

@app.get("/")
async def root():
    return RedirectResponse(url="/demo/")

class Message(BaseModel):
    role: str  # Role can be 'user' or 'assistant'
    content: str  # The content of the message

class ChatRequest(BaseModel):
    model: str  # Model name provided by the client
    system: Optional[str] = None  # Optional system prompt
    messages: List[Message]  # List of messages with roles and content
    temperature: Optional[float] = 0.5  # Optional, default temperature is 0.5
    max_tokens: Optional[int] = 1024  # Optional, default max_tokens is 1024
    stream: Optional[bool] = True  # Enable streaming by default


@app.post("/v1/chat/completions")
def api_chat_completion(chat_request: ChatRequest):
    if not chat_request.messages:
        return {"error": "Messages are required"}

    # 如果 model 是 'psy-1'，調用 SageMaker 端點
    if chat_request.model == "psy-1":
        body = {
            "inputs": f"<|system|>{chat_request.system or ''}<|end|><|user|>{chat_request.messages[-1].content}<|end|><|assistant|>",
            "parameters": {
                "do_sample": True,
                "top_p": 0.9,
                "temperature": chat_request.temperature,
                "max_new_tokens": chat_request.max_tokens,
                "repetition_penalty": 1.03,
                "stop": ["\nUser:", "<|endoftext|>", "###"],
            },
            "stream": chat_request.stream,
        }
        return StreamingResponse(
            sagemaker_stream(
                SAGEMAKER_ENDPOINT_NAME, body
            ),
            media_type="text/html",
        )

    # Default to Bedrock model
    body = {
        "max_tokens": chat_request.max_tokens,  # Accept max_tokens from the front-end
        "anthropic_version": "bedrock-2023-05-31",  # Required by Bedrock API
        "messages": [
            {"role": msg.role, "content": msg.content} for msg in chat_request.messages
        ],
        "temperature": chat_request.temperature,  # Accept temperature from the front-end
    }

    # Include the system prompt if provided
    if chat_request.system:
        body["system"] = chat_request.system

    return StreamingResponse(
        bedrock_stream(chat_request.model, body), media_type="text/html"
    )

async def bedrock_stream(model_id: str, body: dict):
    # Convert the dictionary into a JSON string
    body_str = json.dumps(body)

    # Send the model ID from the request and the body to Bedrock
    response = bedrock.invoke_model_with_response_stream(
        modelId=model_id,  # Model name provided in the request body
        body=body_str,
    )

    stream = response.get("body")
    if stream:
        for event in stream:
            chunk = event.get("chunk")
            if chunk:
                message = json.loads(chunk.get("bytes").decode())
                if message["type"] == "content_block_delta":
                    # Stream the content back to the client
                    yield message["delta"]["text"] or ""
                elif message["type"] == "message_stop":
                    # Indicate the end of the message
                    yield "\n"

async def sagemaker_stream(endpoint_name: str, body: dict):
    body_str = json.dumps(body)

    response = sagemaker_runtime.invoke_endpoint_with_response_stream(
        EndpointName=endpoint_name,
        Body=body_str,
        ContentType='application/json'
    )

    stream = response['Body']
    complete_text = ""  # 用來存儲最終的合併文本
    incomplete_message = ""  # 用來存儲不完整的消息

    if stream:
        for event in stream:
            if 'PayloadPart' in event:
                try:
                    raw_message = event['PayloadPart']['Bytes']
                    # 設定解碼錯誤策略，允許跳過無效字元
                    decoded_message = raw_message.decode('utf-8', errors='ignore')
                    print(f"Raw Message after stripping 'data:': {decoded_message}")

                    if decoded_message.startswith("data:"):
                        decoded_message = decoded_message[5:].strip()

                    # 將不完整的訊息拼接起來
                    if incomplete_message:
                        decoded_message = incomplete_message + decoded_message
                        incomplete_message = ""

                    # 嘗試解析為 JSON
                    try:
                        json_message = json.loads(decoded_message)
                    except json.JSONDecodeError:
                        # 如果 JSON 解析失敗，將訊息暫存並等待下一個 event
                        incomplete_message = decoded_message
                        continue

                    # 確認 token 存在且非特殊字符
                    token = json_message.get('token', {}).get('text')
                    special = json_message.get('token', {}).get('special', False)

                    # 檢查 token 是否是 "<|end|>" 或 special 為 true，直接終止流
                    if token == "<|end|>" or special:
                        print("End token detected, stopping stream.")
                        break  # 結束迴圈並停止拼接

                    if token and token.strip():
                        complete_text += token  # 拼接完整的 text
                        yield token  # 實時回傳 token

                except json.JSONDecodeError:
                    print("JSON Decode Error: Skipping invalid JSON data.")
                    continue

            elif 'ModelStreamError' in event:
                error_message = event['ModelStreamError']['Message']
                print(f"Model stream error: {error_message}")
                # 過濾錯誤訊息，不傳回給前端
                continue

            elif 'InternalStreamFailure' in event:
                print(f"Internal stream failure: {event['InternalStreamFailure']['Message']}")
                # 過濾錯誤訊息，不傳回給前端
                continue



if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=int(os.environ.get("PORT", "8080")))



`run.sh` 用於啟動 Lambda 中的 FastAPI 應用，同時也將會被指定為 Lambda Handler

In [ ]:
%%writefile run.sh

#!/bin/bash

PATH=$PATH:$LAMBDA_TASK_ROOT/bin \
    PYTHONPATH=$PYTHONPATH:/opt/python:$LAMBDA_RUNTIME_DIR \
    exec python -m uvicorn --port=$PORT main:app


In [ ]:
!mkdir -p static

In [ ]:
%%writefile static/index.html

<!DOCTYPE html>
<html>
  <head>
    <title>Psy test demo</title>
    <link rel="stylesheet" href="style.css" />
    <link
      rel="stylesheet"
      href="https://fonts.googleapis.com/css?family=Roboto:300,300italic,700,700italic"
    />
    <link
      rel="stylesheet"
      href="https://cdnjs.cloudflare.com/ajax/libs/normalize/8.0.1/normalize.css"
    />
    <link
      rel="stylesheet"
      href="https://cdnjs.cloudflare.com/ajax/libs/milligram/1.4.1/milligram.css"
    />
  </head>

  <body>
    <div id="container" class="row">
      <div class="column column-67">
        <h1>Psy test demo</h1>
        <h4>
          Enter your request details below and let the AI generate a response.
        </h4>

        <!-- Input fields for model, system prompt, and user message -->
        <label for="model">Model:</label>
        <small
          ><a
            href="https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html"
            target="_blank"
            >Refer to the AWS Bedrock model IDs documentation</a
          ></small
        >
        <input
          type="text"
          id="model"
          placeholder="Enter model ID (e.g., 'anthropic.claude-3-sonnet-20240229-v1:0')"
          value="psy-1"
        />

        <label for="system">System Prompt:</label>
        <textarea id="system" placeholder="Enter system prompt"></textarea>

        <label for="user-message">User Message:</label>
        <textarea id="user-message" placeholder="Enter user message"></textarea>

        <!-- Input for max_tokens and temperature -->
        <label for="max-tokens">Max Tokens:</label>
        <input
          type="number"
          id="max-tokens"
          value="1024"
          min="1"
          placeholder="Enter max tokens"
        />

        <label for="temperature">Temperature:</label>
        <input
          type="number"
          step="0.1"
          id="temperature"
          value="0.5"
          min="0"
          max="1"
          placeholder="Enter temperature"
        />

        <!-- Button to trigger the API call -->
        <button id="generate-response">Generate</button>

        <!-- Output area for the story -->
        <div id="story-output"></div>
      </div>
    </div>

    <script src="script.js"></script>
  </body>
</html>


In [ ]:
%%writefile static/script.js

async function generateAIResponse() {
  // Get input values from the form
  const model = document.getElementById("model").value;
  const system = document.getElementById("system").value;
  const userMessage = document.getElementById("user-message").value;
  const maxTokens = document.getElementById("max-tokens").value;
  const temperature = document.getElementById("temperature").value;

  if (userMessage.trim().length === 0) {
    return;
  }

  const storyOutput = document.getElementById("story-output");
  storyOutput.innerText = "Thinking...";

  try {
    // Create request payload
    const requestBody = {
      model: model,
      system: system,
      messages: [{
        role: 'user',
        content: userMessage
      }],
      max_tokens: parseInt(maxTokens),
      temperature: parseFloat(temperature),
      stream: true
    };

    // Use Fetch API to send a POST request for response streaming
    const response = await fetch("/v1/chat/completions", {
      method: "POST",
      headers: {
        "Content-Type": "application/json"
      },
      body: JSON.stringify(requestBody)
    });

    storyOutput.innerText = "";

    // Response Body is a ReadableStream
    const reader = response.body.getReader();
    const decoder = new TextDecoder();

    // Process the chunks from the stream
    while (true) {
      const {
        done,
        value
      } = await reader.read();
      if (done) {
        break;
      }
      const text = decoder.decode(value);
      storyOutput.innerText += text;
    }

  } catch (error) {
    storyOutput.innerText = `Sorry, an error happened. Please try again later. \n\n ${error}`;
  }
}

document.getElementById("generate-response").addEventListener("click", generateAIResponse);
document.getElementById('user-message').addEventListener('keydown', function (e) {
  if (e.code === 'Enter') {
    generateAIResponse();
  }
});

In [ ]:
%%writefile static/style.css

body {
    font-family: sans-serif;
    margin: 0;
    padding: 0;
  }
  
  #container {
    justify-content: center
  }
  
  h1 {
    text-align: center;
  }
  
  p {
    margin-bottom: 10px;
  }
  
  input {
    width: 100%;
    height: 20px;
    border: 1px solid black;
    margin-bottom: 10px;
  }
  
  button {
    height: 20px;
    background-color: #000;
    color: #fff;
    border: none;
    cursor: pointer;
  }
  
  #story-output {
    width: 100%;
    overflow: auto;
  }
  

### 打包 Lambda 函數為 ZIP 檔案

這段程式碼的目的是將 `main.py`、`run.sh` 以及 `static` 目錄打包成一個 `lambda_package.zip` 檔案，方便在 Lambda 上部署。
1. **創建臨時目錄**：
   - 使用 `os.mkdir` 建立一個臨時目錄 `lambda_temp`，用來存放待打包的檔案。如果目錄已存在，會跳過該步驟。

2. **複製檔案**：
   - 使用 `shutil.copy` 將 `main.py` 和 `run.sh` 複製到臨時目錄。
   - 使用 `shutil.copytree` 將 `static` 資料夾（包含靜態檔案）複製到臨時目錄中。

3. **建立 ZIP 檔案**：
   - 使用 `zipfile.ZipFile` 將臨時目錄中的檔案壓縮為 `lambda_package.zip`，保留目錄結構，使得 Lambda 可以正確找到這些檔案。

4. **清理臨時目錄**：
   - 打包完成後，使用 `shutil.rmtree` 刪除臨時目錄，保持工作空間整潔。

最後，這段程式碼會在當前工作目錄中生成一個 `lambda_package.zip` 檔案，方便後續上傳至 Lambda 進行部署


In [ ]:
import os
import zipfile
import shutil

# Define the names of your files and directories
main_script = 'main.py'
run_script = 'run.sh'
static_dir = 'static'
zip_filename = 'lambda_package.zip'

# Step 1: Create a temporary directory to store the files
if not os.path.exists('lambda_temp'):
    os.mkdir('lambda_temp')

# Step 2: Copy your scripts (main.py and run.sh) into the temp directory
shutil.copy(main_script, './lambda_temp/')
shutil.copy(run_script, './lambda_temp/')

# Step 3: Copy the 'static' directory into the temp directory
if os.path.exists(static_dir):
    shutil.copytree(static_dir, './lambda_temp/static')

# Step 4: Create the .zip package
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk('lambda_temp'):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, arcname=os.path.relpath(file_path, 'lambda_temp'))

# Step 5: Clean up the temp directory
shutil.rmtree('lambda_temp')

print(f"{zip_filename} created successfully.")


# 建立與上傳 FastAPI 相關的 Lambda Layer

這段程式碼用於打包並上傳一個 [Lambda Layer](https://docs.aws.amazon.com/zh_tw/lambda/latest/dg/chapter-layers.html)，該 Layer 包含 FastAPI 及其相關的依賴項，方便 Lambda 使用 FastAPI 框架來處理 HTTP 請求。

<img src="../imgs/lambda-layer.png" alt="Lambda Layer" width="300"/>


In [ ]:
import os
import boto3
import subprocess
import zipfile
import shutil

# 定義變數
layer_dir = "./layer/python"
zip_filename = "layer.zip"
region = sess.boto_region_name
layer_name = "fast_api_related_lambda_layer"
layer_description = "Lambda layer for FastAPI with Lambda Web Adapter"

# Step 1: 建立 layer/python 資料夾
if not os.path.exists(layer_dir):
    os.makedirs(layer_dir)

# Step 2: 安裝指定的套件到 layer/python 資料夾
subprocess.run([
    "pip3", "install", "--target", layer_dir, 
    "annotated-types==0.6.0", 
    "anyio==4.2.0", 
    "click==8.1.7", 
    "exceptiongroup==1.2.0", 
    "fastapi==0.109.2", 
    "h11==0.14.0", 
    "idna==3.7", 
    "pydantic==2.6.1", 
    "pydantic_core==2.16.2", 
    "sniffio==1.3.0", 
    "starlette==0.36.3", 
    "typing_extensions==4.9.0", 
    "uvicorn==0.27.0.post1"
], check=True)

# Step 3: 將 layer 目錄壓縮成 .zip 檔案
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk('./layer'):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, arcname=os.path.relpath(file_path, './layer'))



print(f"{zip_filename} created successfully.")


# 上傳成 Lambda Layer
lambda_client = boto3.client('lambda', region_name=region)

with open(zip_filename, 'rb') as f:
    response = lambda_client.publish_layer_version(
        LayerName=layer_name,
        Description=layer_description,
        Content={'ZipFile': f.read()},
        CompatibleRuntimes=['python3.11'],  # 根據你的 Lambda 版本設定
        LicenseInfo='MIT'  # 可選擇性設定 license
    )

fastapi_related_layer_arn = response["LayerVersionArn"]

# 顯示 Layer ARN
print("Layer uploaded successfully:")
print(fastapi_related_layer_arn)

# 清理 layer 資料夾
shutil.rmtree('./layer')


### 定義 CreateStreamingResponseLambdaFunction Step

我們會使用到 SageMaker 提供的 [Lambda Hepler](https://sagemaker.readthedocs.io/en/stable/api/utility/lambda_helper.html) 類別來創建 Lambda Function
之後使用 [LambdaStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.lambda_step.LambdaStep) 類別來將 Lambda Function 加入到 Pipeline 中

In [ ]:
import time
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput, LambdaOutputTypeEnum
from sagemaker.workflow.functions import JsonGet, Join # https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.functions.JsonGet


streaming_response_function_name_str="lambda_streaming_response_func"


lambda_web_adpter_layer_arn = f"arn:aws:lambda:{sess.boto_region_name}:753240598075:layer:LambdaAdapterLayerX86:23"
pydantic_layer_arn = "arn:aws:lambda:us-west-2:770693421928:layer:Klayers-p311-pydantic:10"


# 創建 Lambda 函數物件
lambda_function = Lambda(
    function_name=streaming_response_function_name_str,
    execution_role_arn=lambda_role,
    zipped_code_dir="lambda_package.zip",
    handler="run.sh",
    timeout=600,
    memory_size=10240,
    runtime="python3.11",
    environment={
        "Variables": {  
            "SAGEMAKER_ENDPOINT_NAME": endpoint_name,
            "AWS_LAMBDA_EXEC_WRAPPER": "/opt/bootstrap",
            "AWS_LWA_INVOKE_MODE": "response_stream",
            "PORT": "8000"
        }
    },
    layers=[lambda_web_adpter_layer_arn, pydantic_layer_arn, fastapi_related_layer_arn]
)

# 創建 Lambda Step
lambda_create_streaming_response_step = LambdaStep(
    name="CreateStreamingResponseLambdaFunction",
    lambda_func=lambda_function,
    depends_on=[lambda_deploy_step]
)


## CreateLambdaFunctionURL Step

在這個步驟中，我們將為之前創建的 Streaming Response Lambda Function 生成一個 [Lambda Function URL](https://docs.aws.amazon.com/lambda/latest/dg/urls-configuration.html)，這樣外部客戶端可以通過這個 URL 訪問 Lambda，並發送推理請求。

### 為什麼選擇 Lambda Function URL？
- 主要原因是 [Lambda Function URL 支援 Streaming Response](https://docs.aws.amazon.com/lambda/latest/dg/response-streaming-tutorial.html)。
- 使用 Lambda Function URL 可以簡化用戶端與 Lambda 之間的互動，不需要經過 API Gateway 的額外配置與流量控制。這對於快速實現 Lambda 與用戶端的直接通信非常有幫助。




In [ ]:
%%writefile lambda_create_function_url.py

import boto3

def lambda_handler(event, context):
    lambda_client = boto3.client('lambda')
    function_name = event['function_name']
    
    # 創建 Lambda Function URL
    response = lambda_client.create_function_url_config(
        FunctionName=function_name,
        AuthType='NONE',
        InvokeMode='RESPONSE_STREAM'
    )
    
    # 返回 Function URL
    return {
        'statusCode': 200,
        'body': {
            'function_url': response['FunctionUrl']
        }
    }

In [ ]:
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput, LambdaOutputTypeEnum


# 創建 Lambda 函數，用於創建 Function URL
lambda_create_function_url = Lambda(
    function_name="create_lambda_function_url",
    execution_role_arn=lambda_role,
    script="lambda_create_function_url.py",
    handler="lambda_create_function_url.lambda_handler",
    timeout=300,
    memory_size=128
)

# 創建 LambdaStep
create_lambda_function_url_step = LambdaStep(
    name="CreateLambdaFunctionURL",
    lambda_func=lambda_create_function_url,
    inputs={
        "function_name": streaming_response_function_name_str
    },
    outputs=[
        LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String),
        LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
    ],
    depends_on=[lambda_create_streaming_response_step]
)


## 定義與創建 SageMaker Pipeline

這段程式碼負責定義整個 SageMaker Pipeline，並將之前設定的所有步驟串聯起來，最後將其創建或更新到 SageMaker 中。

### 主要步驟：
1. **Pipeline 定義**:
   - `Pipeline` 物件中包含 Pipeline 的名稱、參數、以及所有的步驟。
   - `parameters` 定義了我們的 Pipeline 會使用哪些可變的輸入參數，例如訓練數據集的 S3 URI 和推理實例類型。這些參數讓 Pipeline 更加靈活，可以根據不同的需求進行調整。
   - `steps` 列表包含了前面定義的所有步驟，依次為：
     - **train_step**: 模型訓練步驟，fine-tune 預訓練模型。
     - **register_step**: 模型註冊步驟，將訓練完成的模型註冊到 SageMaker Model Registry 中。
     - **lambda_deploy_step**: Lambda 部署步驟，將模型部署到 SageMaker Endpoint。
     - **lambda_create_streaming_response_step**: 創建 Streaming Response Lambda，處理推理請求並返回流式回應。
     - **create_lambda_function_url_step**: 創建 Lambda Function URL，公開 Lambda 讓外部系統可以訪問推理服務。

2. **Pipeline 更新或創建**:
   - 使用 `pipeline.upsert(role_arn=role)` 將 Pipeline 創建或更新到 SageMaker 中。如果這個 Pipeline 已經存在，則會更新；如果不存在，則會創建一個新的。
   - `role_arn` 參數指定了 SageMaker Pipeline 執行所需的 IAM 角色，該角色擁有執行 Pipeline 各步驟的必要權限。

當你執行下方 Code Cell 之後，我們可以到 **SageMaker Studio >  Pipelines** 找到我們所創建的 Pipeline

<img src="../imgs/sagemaker-piepline-list.png" alt="SageMaker Pipeline List" width="300"/>


In [ ]:
# 定義 Pipeline
pipeline = Pipeline(
    name="Parameterize-UncompressedModelArtifact-Deploy-ExposeEndpoint-Pipeline",
    parameters=[inference_instance_type, model_artifact_s3_uri, model_name, random_string],
    steps=[register_step, lambda_deploy_step, lambda_create_streaming_response_step, create_lambda_function_url_step]
)

# 更新 SageMaker Pipeline (不存在則創建)
pipeline.upsert(role_arn=role)


## 執行 SageMaker Pipeline

在這裡我將示範如何在 SageMaker Studio 的頁面中執行我們創建的 Pipeline

打開 SageMaker Studio > Pipelines，點擊要執行的 Pipeline

<img src="../imgs/sagemaker-piepline-list.png" alt="SageMaker Pipeline List" width="400"/>

點擊右上角 **Execute**

- **Execution name:** 20240912-0128
- **Description - optional:** Demo pipeline, Train, Register, Deploy
- **TrainingDatasetesS3Uri (String): <**提供你的 S3 prefix**>**
    - 例如: 你的 `train_datasets.json` ，放在 `s3://sagemaker/trainingdata/` 那這欄位就是填寫 "`s3://sagemaker/trainingdata/`"
- **InferenceInstanceType (String)**: 設定推理所需的實例類型，默認值為 `"ml.g5.xlarge"`

<img src="../imgs/sagemaer-pipeline-execute.png" alt="SageMaker Pipeline Execute" width="400"/>


## 查看 Execution 結果

當我們執行 Pipeline 之後，可以點擊 Executions 頁籤來查看執行的結果

<img src="../imgs/sagemaker-pipeline-execution-list.png" alt="sagemaker-pipeline-execution-list" width="400"/>



我們可以用滑鼠雙擊特定 Step 來查看詳細結果以及 Output，以我下圖為例，我想要找到具體 Function URL ，因此我雙擊 `CreateLambdaFunctionURL` Step，並且從 Output 中找到 `Body`，這是 Lambda Function Return 的 Json Object，裡面包含了我們的 Function URL

<img src="../imgs/sagemaker-pipeline-step-execution-details.png" alt="sagemaker-pipeline-step-execution-details" width="400"/>
